In [97]:
import random
import simpy
import numpy as np
import scipy.stats as st


In [98]:
RANDOM_SEED = 42
TASK_MEAN = 24.0
# TASK_SIGMA = 2.0

INTERRUPTION_MEAN = 10.0
INTERRUPTION_SIGMA = 3.0

BREAK_MEAN = 6.0


random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [99]:
def time_per_task():
    return st.expon.rvs(size=1, scale=TASK_MEAN)[0]

def time_to_interrupt():
    t = random.normalvariate(INTERRUPTION_MEAN, INTERRUPTION_SIGMA)

    while t <= 0:
        t = random.normalvariate(INTERRUPTION_MEAN, INTERRUPTION_SIGMA)
    return t

def time_to_break():
    return time_per_task()

def break_duration():
    return st.expon.rvs(size=1, scale=BREAK_MEAN)[0]

def interruption_duration():
    return st.expon.rvs(size=1, scale=INTERRUPTION_SIGMA)[0]



In [104]:



class Person:
    def __init__(self,env,state):
        self.state = state
        self.env = env
        self.person = simpy.PreemptiveResource(env,capacity=1)
        self.completed_tasks = 0
        self.breaks = 0
        self.interrupts = 0

        self.process_working = env.process(self.working())
        # env.process(self.interrupting())
        self.process_break = env.process(self.take_break())

    def working(self):
        while True:
            time = time_per_task()
            # print('duracion de la proxima tarea', time)
            
            while time:    
                start = self.env.now
                try:
                    print(f'Minuto {self.env.now} | iniciando tarea {self.completed_tasks+1} | queda por trabajar {time} min')
                    self.state = 'W'

                    yield self.env.timeout(time)
                    time = 0
                    self.state = 'S'

                except simpy.Interrupt:
                    time = max(0, time + start - self.env.now)
                    
                    if self.state == 'I':
                        interruption_time = interruption_duration()
                        print(f'Minuto {self.env.now} | trabajo interrumpido por {interruption_time} | quedando por completar unos {time} min de tarea')
                        
                        yield self.env.timeout(interruption_time)
                        print(f'Minuto {self.env.now} | interrupción terminada')
                        self.interrupts += 1
                        #self.state = 'W'
                        
                    elif self.state == 'D': 
                        break_time = break_duration()
                        start_break = self.env.now
                        print(f'Minuto {self.env.now} | descanso por {break_time} min | quedando por completar unos {time} min de tarea')

                        while break_time:
                            try:
                                yield self.env.timeout(break_time)
                                break_time = 0
                                #self.state = 'W'

                            except:
                                break_time = max(0, break_time + start_break - self.env.now)
                                interruption_time = interruption_duration()
                                print(f'Minuto {self.env.now} | descanso interrumpido por {interruption_time} min')
                                yield self.env.timeout(interruption_time)
                                print(f'Minuto {self.env.now} | interrupción terminada')
                                self.interrupts += 1
                                yield self.env.timeout(break_time)
                                self.state = 'D'

                            self.breaks += 1
                            print(f'Minuto {self.env.now} | descanso completada en el minuto', self.env.now)
                                
                            

                    yield self.env.timeout(time)
                    time = 0
                    self.state = 'S'            
                    
            self.completed_tasks +=1
            print(f'Minuto {self.env.now} | tarea numero {self.completed_tasks} completada')

    def take_break(self):
        while True:
            time = time_to_break()
            print('solicitando el proximo descanso en ',self.env.now, ' para dentro de ', time)
            yield self.env.timeout(time)
           
            with self.person.request(priority=1) as request:
                yield request
                    
                if self.state == 'W':
                    self.state = 'D'
                    self.process_working.interrupt()
                elif self.state == 'I':
                    print('no puedes descansar por interrupcion', self.env.now)
                

    def interrupting(self):
            while True:
                time = time_to_interrupt()
                print('solicitando proxima interrupcion en ', self.env.now, ' para dentro de :', time)
                yield self.env.timeout(time)
                
                with self.person.request(priority=0) as request:
                    yield request
                    print('se obtuvo la interrupcion (el recurso) en el minuto', self.env.now)
                    if self.state == 'W':
                        self.state = 'I'
                        self.process_working.interrupt()
                    elif self.state == 'D':
                        self.state = 'I'
                        self.process_working.interrupt()
                    else:
                        print(f'no se puede interrumpir en el minuto', self.env.now)
                        continue
            


In [105]:
random.seed(RANDOM_SEED)

env = simpy.Environment()
person = Person(env,'S')
env.run(until=200)
print(person.breaks)
print(person.completed_tasks)
print(person.interrupts)

Minuto 0 | iniciando tarea 1 | queda por trabajar 5.728503005974728 min
solicitando el proximo descanso en  0  para dentro de  4.8162957299919205
solicitando el proximo descanso en  4.8162957299919205  para dentro de  4.862674147974169
Minuto 4.8162957299919205 | descanso por 2.176522376762863 min | quedando por completar unos 0.9122072759828077 min de tarea
Minuto 6.992818106754783 | descanso completada en el minuto 6.992818106754783
Minuto 7.905025382737591 | tarea numero 1 completada
Minuto 7.905025382737591 | iniciando tarea 2 | queda por trabajar 17.854267940660506 min
solicitando el proximo descanso en  9.67896987796609  para dentro de  13.572889602728079
Minuto 9.67896987796609 | descanso por 2.065337955323649 min | quedando por completar unos 16.080323445432008 min de tarea
Minuto 11.744307833289739 | descanso completada en el minuto 11.744307833289739
solicitando el proximo descanso en  23.25185948069417  para dentro de  22.71290097359517
Minuto 27.824631278721746 | tarea nume